In [1]:
# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

In [2]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [3]:
import os
import sys
import pandas as pd
import numpy as np
import tqdm
import glob
import shutil

import pydicom
import nibabel as nib
import SimpleITK as sitk

import copy
pd.set_option('max_colwidth', -1)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

In [4]:
def sanity_check_volume_slices(dir_path):
    file_paths= glob.glob(dir_path + '*.nii')
    volume_data = []
    
    for fullname in file_paths:
        nii_obj = nib.load(fullname)
        (nx, ny, nz) = nii_obj.shape[0:3]
        (sx, sy, sz) = nii_obj.header.get_zooms()    #  Slice thickness
        voxel_spacing = sx * sy* sz
        filename = fullname.split(os.sep)[-1]
        data = [filename, nx, ny, nz, voxel_spacing]
        volume_data.append(data)

    return volume_data

def max_num_slice(input_path):
    num_slices = []
        
    for volume_path in sorted(glob.glob(os.path.join(input_path,'*.nii'))):

        volume_obj= nib.load(volume_path)
        volume_data= volume_obj.get_fdata()
        x, y, z = volume_data.shape
        num_slices.append(z)
    
    return num_slices, max(num_slices)



def volume_depth_correction(input_path, output_path):
    _, max_slices = max_num_slice(input_path)

    for volume_path in sorted(glob.glob(os.path.join(input_path,'*.nii'))):
        volume_obj= nib.load(volume_path)
        volume_data= volume_obj.get_fdata()
        x, y, z = volume_data.shape

        num_slices, max_slice= max_num_slice(input_path)
    
        if z == max_slice:
            print(volume_path)
            volume_name= os.path.split(volume_path)[-1]
            print(volume_name)
            shutil.copy(volume_path, os.path.join(output_path, volume_name))

        if z < max_slice:
            sx, sy, sz = volume_obj.header.get_zooms()
            voxel_volume = sx * sy * sz
            
            print("processing original voume: ", volume_path, x, y, z, voxel_volume)
            print(max_slice)
            last_slice = volume_data[:, :, z-1]

            black_slice = copy.deepcopy(last_slice)
            black_slice[:,:] = 0

            delta_slices = (max_slices - z)
            print("delta", delta_slices)
            b = np.dstack([black_slice]*(delta_slices))
            volume_depth_corrected = np.dstack((volume_data, b))
            
            final_volume = nib.Nifti1Image(volume_depth_corrected, volume_obj.affine, header=volume_obj.header)
            volume_name = os.path.split(volume_path)[-1]
            nib.save(final_volume, os.path.join(output_path, volume_name))       

            volume_corrected_data = final_volume.get_fdata()
            x, y, z = volume_corrected_data.shape

            sx, sy, sz = final_volume.header.get_zooms()
            voxel_volume_corrected = sx * sy * sz

            print("corrected volume:", x, y, z, voxel_volume_corrected)
            print("********************")

In [5]:
data_dir= 'cynch_seg_seg/'
print(os.listdir(data_dir))

['human_corrected_without_vessel_removal', 'human_corrected_with_vessel_removal', 'images']


In [6]:
img_dir= os.path.join(data_dir, 'images/')
mask_vessel= os.path.join(data_dir, 'human_corrected_with_vessel_removal/')
mask_novessel= os.path.join(data_dir, 'human_corrected_without_vessel_removal/')

In [7]:
img_files = sorted(glob.glob(os.path.join(img_dir, '*.nii')))
mask_vessel_files = sorted(glob.glob(os.path.join(mask_vessel, '*.nii')))
mask_novessel_files = sorted(glob.glob(os.path.join(mask_novessel, '*.nii')))

In [8]:
print(f'Total # of images: {len(img_files)}' )
print(f' Total # of masks, no vessel: {len(mask_novessel_files)}')
print(f'Total #  of masks, with vessel: {len(mask_vessel_files)}')

Total # of images: 111
 Total # of masks, no vessel: 111
Total #  of masks, with vessel: 111


In [9]:
volume_data = sanity_check_volume_slices(img_dir)
df = pd.DataFrame(volume_data, columns= ['volume', 'width', 'height', '#_slices', 'voxel_spacing'])
print(df)

         volume  width  height  #_slices  voxel_spacing
0    c_1309.nii  224    224     30        38.586430    
1    c_1326.nii  224    224     20        44.025036    
2    c_1370.nii  224    224     28        44.025036    
3    c_1445.nii  224    224     22        31.887756    
4    c_1453.nii  224    224     24        44.025036    
5    c_1459.nii  224    224     23        28.779669    
6    c_1464.nii  224    224     24        44.025036    
7    c_1481.nii  224    224     24        44.025036    
8    c_1493.nii  224    224     24        44.025036    
9    c_1501.nii  224    224     30        44.025036    
10   c_1509.nii  224    224     24        44.025036    
11   c_1512.nii  224    224     24        44.025036    
12   c_1513.nii  224    224     25        44.025036    
13   c_1514.nii  224    224     24        35.156250    
14   c_2281.nii  224    224     21        23.915815    
15   c_2288.nii  224    224     22        23.915815    
16   c_2351.nii  224    224     22        23.915

In [10]:
input_path= 'cynch_seg_seg/images/'
output_path= 'data_corrected/images'


volume_depth_correction(input_path, output_path)

processing original voume:  cynch_seg_seg/images\c_1309.nii 224 224 30 38.58643
35
delta 5
corrected volume: 224 224 35 38.58643
********************
processing original voume:  cynch_seg_seg/images\c_1326.nii 224 224 20 44.025036
35
delta 15
corrected volume: 224 224 35 44.025036
********************
processing original voume:  cynch_seg_seg/images\c_1370.nii 224 224 28 44.025036
35
delta 7
corrected volume: 224 224 35 44.025036
********************
processing original voume:  cynch_seg_seg/images\c_1445.nii 224 224 22 31.887756
35
delta 13
corrected volume: 224 224 35 31.887756
********************
processing original voume:  cynch_seg_seg/images\c_1453.nii 224 224 24 44.025036
35
delta 11
corrected volume: 224 224 35 44.025036
********************
processing original voume:  cynch_seg_seg/images\c_1459.nii 224 224 23 28.779669
35
delta 12
corrected volume: 224 224 35 28.779669
********************
processing original voume:  cynch_seg_seg/images\c_1464.nii 224 224 24 44.025036
35
de

processing original voume:  cynch_seg_seg/images\c_5429.nii 224 224 34 28.125
35
delta 1
corrected volume: 224 224 35 28.125
********************
processing original voume:  cynch_seg_seg/images\c_5430.nii 224 224 20 25.509794
35
delta 15
corrected volume: 224 224 35 25.509794
********************
processing original voume:  cynch_seg_seg/images\c_5431.nii 224 224 20 25.509794
35
delta 15
corrected volume: 224 224 35 25.509794
********************
processing original voume:  cynch_seg_seg/images\c_5434.nii 224 224 32 28.125
35
delta 3
corrected volume: 224 224 35 28.125
********************
cynch_seg_seg/images\c_5435.nii
c_5435.nii
processing original voume:  cynch_seg_seg/images\c_5437.nii 224 224 29 23.022184
35
delta 6
corrected volume: 224 224 35 23.022184
********************
processing original voume:  cynch_seg_seg/images\c_5438.nii 224 224 29 23.022184
35
delta 6
corrected volume: 224 224 35 23.022184
********************
processing original voume:  cynch_seg_seg/images\c_5440

In [12]:
input_path= 'cynch_seg_seg/human_corrected_without_vessel_removal/'
output_path= 'data_corrected/human_corrected_without_vessel_removal'


volume_depth_correction(input_path, output_path)

processing original voume:  cynch_seg_seg/human_corrected_without_vessel_removal\c_1309.nii 224 224 30 38.58643
35
delta 5
corrected volume: 224 224 35 38.58643
********************
processing original voume:  cynch_seg_seg/human_corrected_without_vessel_removal\c_1326.nii 224 224 20 44.025036
35
delta 15
corrected volume: 224 224 35 44.025036
********************
processing original voume:  cynch_seg_seg/human_corrected_without_vessel_removal\c_1370.nii 224 224 28 44.025036
35
delta 7
corrected volume: 224 224 35 44.025036
********************
processing original voume:  cynch_seg_seg/human_corrected_without_vessel_removal\c_1445.nii 224 224 22 31.887756
35
delta 13
corrected volume: 224 224 35 31.887756
********************
processing original voume:  cynch_seg_seg/human_corrected_without_vessel_removal\c_1453.nii 224 224 24 44.025036
35
delta 11
corrected volume: 224 224 35 44.025036
********************
processing original voume:  cynch_seg_seg/human_corrected_without_vessel_remova

processing original voume:  cynch_seg_seg/human_corrected_without_vessel_removal\c_5348.nii 224 224 20 25.509794
35
delta 15
corrected volume: 224 224 35 25.509794
********************
processing original voume:  cynch_seg_seg/human_corrected_without_vessel_removal\c_5378.nii 224 224 28 30.867798
35
delta 7
corrected volume: 224 224 35 30.867798
********************
processing original voume:  cynch_seg_seg/human_corrected_without_vessel_removal\c_5413.nii 224 224 30 25.509794
35
delta 5
corrected volume: 224 224 35 25.509794
********************
processing original voume:  cynch_seg_seg/human_corrected_without_vessel_removal\c_5421.nii 224 224 32 30.868246
35
delta 3
corrected volume: 224 224 35 30.868246
********************
processing original voume:  cynch_seg_seg/human_corrected_without_vessel_removal\c_5423.nii 224 224 20 25.509794
35
delta 15
corrected volume: 224 224 35 25.509794
********************
processing original voume:  cynch_seg_seg/human_corrected_without_vessel_remov

processing original voume:  cynch_seg_seg/human_corrected_without_vessel_removal\c_5589.nii 224 224 26 20.662163
35
delta 9
corrected volume: 224 224 35 20.662163
********************
processing original voume:  cynch_seg_seg/human_corrected_without_vessel_removal\c_5593.nii 224 224 29 25.509794
35
delta 6
corrected volume: 224 224 35 25.509794
********************
processing original voume:  cynch_seg_seg/human_corrected_without_vessel_removal\c_6080.nii 224 224 24 44.025036
35
delta 11
corrected volume: 224 224 35 44.025036
********************
processing original voume:  cynch_seg_seg/human_corrected_without_vessel_removal\c_7353.nii 224 224 28 25.268555
35
delta 7
corrected volume: 224 224 35 25.268555
********************
processing original voume:  cynch_seg_seg/human_corrected_without_vessel_removal\c_8728.nii 224 224 17 26.90529
35
delta 18
corrected volume: 224 224 35 26.90529
********************
processing original voume:  cynch_seg_seg/human_corrected_without_vessel_removal

In [13]:
input_path= 'cynch_seg_seg/human_corrected_with_vessel_removal/'
output_path= 'data_corrected/human_corrected_with_vessel_removal'


volume_depth_correction(input_path, output_path)

processing original voume:  cynch_seg_seg/human_corrected_with_vessel_removal\c_1309.nii 224 224 30 38.58643
35
delta 5
corrected volume: 224 224 35 38.58643
********************
processing original voume:  cynch_seg_seg/human_corrected_with_vessel_removal\c_1326.nii 224 224 20 44.025036
35
delta 15
corrected volume: 224 224 35 44.025036
********************
processing original voume:  cynch_seg_seg/human_corrected_with_vessel_removal\c_1370.nii 224 224 28 44.025036
35
delta 7
corrected volume: 224 224 35 44.025036
********************
processing original voume:  cynch_seg_seg/human_corrected_with_vessel_removal\c_1445.nii 224 224 22 31.887756
35
delta 13
corrected volume: 224 224 35 31.887756
********************
processing original voume:  cynch_seg_seg/human_corrected_with_vessel_removal\c_1453.nii 224 224 24 44.025036
35
delta 11
corrected volume: 224 224 35 44.025036
********************
processing original voume:  cynch_seg_seg/human_corrected_with_vessel_removal\c_1459.nii 224 2

processing original voume:  cynch_seg_seg/human_corrected_with_vessel_removal\c_5378.nii 224 224 28 30.867798
35
delta 7
corrected volume: 224 224 35 30.867798
********************
processing original voume:  cynch_seg_seg/human_corrected_with_vessel_removal\c_5413.nii 224 224 30 25.509794
35
delta 5
corrected volume: 224 224 35 25.509794
********************
processing original voume:  cynch_seg_seg/human_corrected_with_vessel_removal\c_5421.nii 224 224 32 30.868246
35
delta 3
corrected volume: 224 224 35 30.868246
********************
processing original voume:  cynch_seg_seg/human_corrected_with_vessel_removal\c_5423.nii 224 224 20 25.509794
35
delta 15
corrected volume: 224 224 35 25.509794
********************
processing original voume:  cynch_seg_seg/human_corrected_with_vessel_removal\c_5424.nii 224 224 33 23.022184
35
delta 2
corrected volume: 224 224 35 23.022184
********************
processing original voume:  cynch_seg_seg/human_corrected_with_vessel_removal\c_5425.nii 224 2

processing original voume:  cynch_seg_seg/human_corrected_with_vessel_removal\c_6080.nii 224 224 24 44.025036
35
delta 11
corrected volume: 224 224 35 44.025036
********************
processing original voume:  cynch_seg_seg/human_corrected_with_vessel_removal\c_7353.nii 224 224 28 25.268555
35
delta 7
corrected volume: 224 224 35 25.268555
********************
processing original voume:  cynch_seg_seg/human_corrected_with_vessel_removal\c_8728.nii 224 224 17 26.90529
35
delta 18
corrected volume: 224 224 35 26.90529
********************
processing original voume:  cynch_seg_seg/human_corrected_with_vessel_removal\c_8733.nii 224 224 22 25.908798
35
delta 13
corrected volume: 224 224 35 25.908798
********************
processing original voume:  cynch_seg_seg/human_corrected_with_vessel_removal\c_8740.nii 224 224 31 25.908798
35
delta 4
corrected volume: 224 224 35 25.908798
********************
processing original voume:  cynch_seg_seg/human_corrected_with_vessel_removal\c_9110.nii 224 2

In [11]:
# os.listdir('data_corrected/')

['human_corrected_without_vessel_removal',
 'human_corrected_with_vessel_removal',
 'images']